In [9]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [10]:
def style_content_lose(outputs):
    style_outputs = outputs['style'] #用来表示style信息的网络层输出，这里已经计算过Gram矩阵了，已经除过NM
    content_outputs = outputs['content'] #用来表示content信息的网络层输出，不需要Gram矩阵
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                          for name in style_outputs.keys()])
    style_loss *= style_weight / num_style_layers
    
    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                            for name in content_outputs.keys()])
    content_loss *= content_weight / num_content_layers
    loss = style_loss + content_loss
    return loss

In [11]:
def gram_matrix(input_tensor):
    result = tf.einsum('lijc,lijd->lcd',input_tensor,input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1]*input_shape[2],tf.float32)
    return result/(num_locations)

In [12]:
def load_img(path_to_img):
    max_dim=1200
    img = tf.io.read_file(path_to_img) #读取的还是乱码
    img = tf.image.decode_image(img,channels=3) #数值为0-256的RGB
    img = tf.image.convert_image_dtype(img,tf.float32) #数值变为0-1
    
    shape = tf.cast(tf.shape(img)[:-1],tf.float32)
    long_dim = max(shape)
    scale = max_dim/long_dim
    
    new_shape = tf.cast(shape*scale,tf.int32) #图片长宽最高为1200
    
    img = tf.image.resize(img,new_shape)
    img = img[tf.newaxis,:]
    
    return img

In [13]:
def clip_0_1(image):
    return tf.clip_by_value(image,clip_value_min=0.0,clip_value_max=1.0)

In [14]:
def high_pass_x_y(image):
    x_var = image[:,:,1:,:]-image[:,:,:-1,:]
    y_var = image[:,1:,:,:]-image[:,:-1,:,:]
    return x_var, y_var

In [15]:
def total_variation_loss(image):
    x_deltas,y_deltas = high_pass_x_y(image)
    return tf.reduce_mean(x_deltas**2)+tf.reduce_mean(y_deltas**2)